In [ ]:
import CalSim as cs
import numpy as np
from controllers.force_controllers import *
from trajectories import *
import matplotlib.pyplot as plt
import time
import trimesh
import csv

def parse_csv_to_positions(file_path):
    positions = []
    with open(file_path, newline='', encoding='latin-1') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            start = [float(row['start_x']), float(row['start_y']), float(row['start_z'])]
            goal = [float(row['goal_x']), float(row['goal_y']), float(row['goal_z'])]
            positions.append((start, goal))
    return positions


file_path = '/scratch/rhm4nj/cral/cral-ginn/ginn/goal_csvs/room_0_start_goal.csv'
position_pairs = parse_csv_to_positions(file_path)
traj_counter = 1



for start, goal in position_pairs:
    traj_start = time.time()
    print(start)
    print(goal)
    #system initial condition
    pos0 = np.array([start]).T
    vel0 = np.zeros((3, 1))
    omega0 = np.zeros((3, 1))
    R0 = np.eye(3).reshape((9, 1))
    x0 = np.vstack((pos0, R0, omega0, vel0))
    print(x0)

    #create a dynamics object for the double integrator
    dynamics = cs.Qrotor3D(x0)

    #create an observer
    observerManager = cs.ObserverManager(dynamics)

    #create an obstacle - old code
    # qObs = np.array([[0, 1, 1], [0, 0.5, 2]]).T
    # rObs = [0.25, 0.25]
    # obstacleManager = cs.ObstacleManager(qObs, rObs, NumObs = 2)

    #OWN METHOD:
    #nx3 pointcloud, radius of point, number of obstacles 
    #for each of the points in pointcloud create an obstacle 
    def points_to_obstacles(point_cloud_arr, radius):
        qObs = point_cloud_arr.T
        rObs = [radius] * point_cloud_arr.shape[0]
        numObs = point_cloud_arr.shape[0]
        print(numObs)
        return cs.ObstacleManager(qObs, rObs, NumObs = numObs)

    # set data_path
    mesh = trimesh.load_mesh("/scratch/rhm4nj/cral/datasets/Replica-Dataset/ReplicaSDK/room_0/mesh.ply")
    coord = np.array(mesh.vertices)

    obstacleManager = points_to_obstacles(coord, 0.1)

    #create a depth camera
    depthManager = cs.DepthCamManager(observerManager, obstacleManager, mean = None, sd = None)
    # print(depthManager.get_depth_cam_i(0).calc_ptcloud_world())
    # print(depthManager.get_depth_cam_i(0).calc_ptcloud_world().shape)

    #visualize point cloud with k3d
    # point_cloud = np.transpose(depthManager.get_depth_cam_i(0).calc_ptcloud_world())
    # import k3d

    # plot = k3d.plot()
    # points = k3d.points(point_cloud, point_size=0.1)

    # plot += points
    # plot.display()


    #create a trajectory
    xD = np.vstack((np.array([goal]).T, R0, omega0, vel0))
    print(xD)
    trajManager = TrajectoryManager(x0, xD, Ts = 5, N = 1)

    #create a controller manager with a basic FF controller
    controllerManager = cs.ControllerManager(observerManager, QRotorGeometricPD, None, trajManager, depthManager)

    #create an environment
    env = cs.Environment(dynamics, controllerManager, observerManager, obstacleManager, T = 10)
    env.reset()

    #run the simulation
    anim = env.run()
    traj_end = time.time()

    trajectory = anim[0][0:3, :].T
    # trajectory = np.zeros((18,51))
    np.savetxt(f"/home/nqd2xs/depth_cbf/qrotor_sim_examples/room0_trajectories/trajectory{traj_counter}.txt", trajectory, delimiter=",", header=f"Execution time: {traj_end - traj_start:.4f} seconds")
    traj_counter += 1
 


[5.874838419278594, 2.7941896761580045, -1.1]
[0.1382249543385821, 1.7850875121885492, -1.1]
[[ 5.87483842]
 [ 2.79418968]
 [-1.1       ]
 [ 1.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
954491
[[ 0.13822495]
 [ 1.78508751]
 [-1.1       ]
 [ 1.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
No barrier/lyapunov object passed in.
Running Simulation.


KeyboardInterrupt: 